In [3]:
import mysql_connection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import joblib


def fetch_data(teams):
    connection = mysql_connection.get_mysql_connection()
    if connection:
        if teams == 'All':
            sql_query = """
            SELECT team_1, team_2, winner, margin, ground, match_date, match_id, team_1_score, team_1_wicket, team_1_over, team_2_score, team_2_wicket, team_2_over, team_1_total_over, team_2_total_over, first_bat_team, second_bat_team
            FROM cricket_info.matches;
            """
            cursor = connection.cursor()
            cursor.execute(sql_query)
        else:
            sql_query = """
            SELECT team_1, team_2, winner, margin, ground, match_date, match_id, 
                   team_1_score, team_1_wicket, team_1_over, team_2_score, team_2_wicket, 
                   team_2_over, team_1_total_over, team_2_total_over, first_bat_team, 
                   second_bat_team
            FROM cricket_info.matches
            WHERE team_1 IN ({}) OR team_2 IN ({});
            """.format(','.join(['%s']*len(teams)), ','.join(['%s']*len(teams)))

            cursor = connection.cursor()
            cursor.execute(sql_query, teams*2)
           
        rows = cursor.fetchall()
        df = pd.DataFrame(rows, columns=['team_1', 'team_2', 'winner', 'margin', 'ground', 'match_date', 'match_id', 'team_1_score', 'team_1_wicket', 'team_1_over', 'team_2_score', 'team_2_wicket', 'team_2_over', 'team_1_total_over', 'team_2_total_over', 'first_bat_team', 'second_bat_team'])
        cursor.close()
        connection.close()

        # Define the list of ICC T20 teams
        icc_t20_teams = ["Afghanistan", "Australia", "Bangladesh", "England", "India", "New Zealand", "Pakistan", "South Africa", "Sri Lanka", "West Indies"]
        
        # Filter the DataFrame based on the conditions
        df = df[(df['team_1'].isin(icc_t20_teams)) & (df['team_2'].isin(icc_t20_teams))]

        
        return df
    else:
        print("Connection to the database failed.")
        return None

# Function to convert string representation of overs to balls 
def overs_to_balls(overs_str):
    # Split overs into integer and fractional parts
    parts = str(overs_str).split('.')
    
    # If only integer part is present
    if len(parts) == 1:
        overs_int = int(parts[0])
        overs_frac = 0
    # If both integer and fractional parts are present
    elif len(parts) == 2:
        overs_int = int(parts[0])
        overs_frac = int(parts[1])
    else:
        raise ValueError("Invalid overs string format")

    # Convert overs to balls and add fractional part
    balls = overs_int * 6 + overs_frac
    
    return balls


def transform_data(df):

    # Create new columns to store wickets and runs margins
    df['wickets_margin'] = 0
    df['runs_margin'] = 0
    
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        margin = row['margin']
        if 'wickets' in margin:
            # Extract the number for wickets margin
            wickets_margin = int(margin.split()[0])
            df.at[index, 'wickets_margin'] = wickets_margin
            df.at[index, 'runs_margin'] = 0

        elif 'runs' in margin:
            # Extract the number for runs margin
            runs_margin = int(margin.split()[0])
            df.at[index, 'runs_margin'] = runs_margin
            df.at[index, 'wickets_margin'] = 0


        else:
            df.at[index, 'wickets_margin'] = 0
            df.at[index, 'runs_margin'] = 0

    df['date'] = pd.to_datetime(df['match_date'], format='%b %d, %Y', errors='coerce')
    df['team_1_score'] = pd.to_numeric(df['team_1_score'], errors='coerce')
    df['team_1_wicket'] = pd.to_numeric(df['team_1_wicket'], errors='coerce')
    df['team_1_over'] = df['team_1_over'].apply(overs_to_balls)
    df['team_2_score'] = pd.to_numeric(df['team_2_score'], errors='coerce')
    df['team_2_wicket'] = pd.to_numeric(df['team_2_wicket'], errors='coerce')
    df['team_2_over'] = df['team_2_over'].apply(overs_to_balls)
    df['team_1_total_over'] = df['team_1_total_over'].apply(overs_to_balls)
    df['team_2_total_over'] = df['team_2_total_over'].apply(overs_to_balls)

    df.drop(columns=['margin'], inplace=True)
    df.drop(columns=['match_id'], inplace=True)
    df.drop(columns=['match_date'], inplace=True)


    print(df)
    print(df.dtypes)

    return df


from sklearn.metrics import accuracy_score, mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# def models_evaluate(df):
#     master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

#     # Ensure 'team_1', 'team_2', 'winner', and 'ground' columns are of string data type
#     master_df[['team_1', 'team_2', 'winner', 'ground', 'first_bat_team', 'second_bat_team']] = master_df[['team_1', 'team_2', 'winner', 'ground', 'first_bat_team', 'second_bat_team']].astype(str)
    
#     # Define features for prediction
#     numeric_features = ['team_1_score', 'team_1_wicket', 'team_1_over', 'team_2_score', 'team_2_wicket', 'team_2_over', 'team_1_total_over', 'team_2_total_over', 'wickets_margin', 'runs_margin']
    
#     # Splitting the data into features (X) and target variable (y)
#     X = master_df[numeric_features]
#     y = master_df['winner']
    
#     # Splitting the data into training and testing sets
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     # Preprocessing: Define numeric transformer
#     numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    
#     preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features)])
    
#     # Initialize classification models
#     models = [
#         ('Logistic Regression', LogisticRegression(max_iter=1000)),
#         ('Random Forest Classifier', RandomForestClassifier(random_state=42)),
#         ('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=42)),
#         ('Decision Tree Classifier', DecisionTreeClassifier(random_state=42)),
#         ('Support Vector Classifier', SVC()),
#     ]

#     # Define parameter grids for models
#     param_grids = {
#         'Logistic Regression': {'classifier__C': [0.1, 1, 10]},
#         'Random Forest Classifier': {'classifier__n_estimators': [100, 200, 300]},
#         'Gradient Boosting Classifier': {'classifier__n_estimators': [50, 100, 150], 'classifier__learning_rate': [0.01, 0.1, 0.2]},
#         'Decision Tree Classifier': {'classifier__max_depth': [None, 5, 10, 20]},
#         'Support Vector Classifier': {'classifier__C': [0.1, 1, 10], 'classifier__gamma': [0.1, 1, 10]}
#     }

#     # Initialize classification models
#     models = [
#         ('Logistic Regression', LogisticRegression(max_iter=1000)),
#         ('Random Forest Classifier', RandomForestClassifier(random_state=42)),
#         ('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=42)),
#         ('Decision Tree Classifier', DecisionTreeClassifier(random_state=42)),
#         ('Support Vector Classifier', SVC()),
#     ]
    
#     # Train and evaluate models with parameter tuning and cross-validation
#     for name, model in models:
#         classifier = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
#         param_grid = param_grids.get(name, {})  # Get parameter grid for the current model
#         stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#         grid_search = GridSearchCV(classifier, param_grid, cv=stratified_cv, scoring='accuracy')
#         grid_search.fit(X_train, y_train)
#         best_model = grid_search.best_estimator_
#         predictions = best_model.predict(X_test)
#         accuracy = accuracy_score(y_test, predictions)
#         print(f'{name}:')
#         print(f'Best Parameters: {grid_search.best_params_}')
#         print(f'Accuracy: {accuracy}')
#         print('\n')


from sklearn.metrics import f1_score

def models_evaluate(df):
    master_df = df.copy()  # Always work on a copy to avoid unintended modifications to the original DataFrame

    # Ensure 'team_1', 'team_2', 'winner', and 'ground' columns are of string data type
    master_df[['team_1', 'team_2', 'winner', 'ground', 'first_bat_team', 'second_bat_team']] = master_df[['team_1', 'team_2', 'winner', 'ground', 'first_bat_team', 'second_bat_team']].astype(str)
    
    # Define features for prediction
    numeric_features = ['team_1_score', 'team_1_wicket', 'team_1_over', 'team_2_score', 'team_2_wicket', 'team_2_over', 'team_1_total_over', 'team_2_total_over', 'wickets_margin', 'runs_margin']
    
    # Splitting the data into features (X) and target variable (y)
    X = master_df[numeric_features]
    y = master_df['winner']
    
    # Splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric transformer
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features)])
    
    # Initialize classification models
    models = [
        ('Logistic Regression', LogisticRegression(max_iter=1000)),
        ('Random Forest Classifier', RandomForestClassifier(random_state=42)),
        ('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=42)),
        ('Decision Tree Classifier', DecisionTreeClassifier(random_state=42)),
        ('Support Vector Classifier', SVC()),
    ]

    # Define parameter grids for models
    param_grids = {
        'Logistic Regression': {'classifier__C': [0.1, 1, 10]},
        'Random Forest Classifier': {'classifier__n_estimators': [100, 200, 300]},
        'Gradient Boosting Classifier': {'classifier__n_estimators': [50, 100, 150], 'classifier__learning_rate': [0.01, 0.1, 0.2]},
        'Decision Tree Classifier': {'classifier__max_depth': [None, 5, 10, 20]},
        'Support Vector Classifier': {'classifier__C': [0.1, 1, 10], 'classifier__gamma': [0.1, 1, 10]}
    }

    # Train and evaluate models with parameter tuning and cross-validation
    for name, model in models:
        classifier = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', model)])
        param_grid = param_grids.get(name, {})  # Get parameter grid for the current model
        stratified_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(classifier, param_grid, cv=stratified_cv, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        predictions = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        f1 = f1_score(y_test, predictions, average='weighted')
        print(f'{name}:')
        print(f'Best Parameters: {grid_search.best_params_}')
        print(f'Accuracy: {accuracy}')
        print(f'F1 Score: {f1}')
        print('\n')



def model_implement(df):
    master_df = df.copy()

    # Ensure 'team_1', 'team_2', 'winner', and 'ground' columns are of string data type
    master_df[['team_1', 'team_2', 'winner', 'ground', 'first_bat_team', 'second_bat_team']] = master_df[['team_1', 'team_2', 'winner', 'ground', 'first_bat_team', 'second_bat_team']].astype(str)
    
    # Define features for prediction
    numeric_features = ['team_1_score', 'team_1_wicket', 'team_1_over', 'team_2_score', 'team_2_wicket', 'team_2_over', 'team_1_total_over', 'team_2_total_over', 'wickets_margin', 'runs_margin']
    
    # Splitting the data into features (X) and target variable (y)
    X = master_df[numeric_features]
    y = master_df['winner']
    
    # Splitting the data into training and testing sets
    X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Preprocessing: Define numeric transformer
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())])
    preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, numeric_features)])
    
    # Initialize and train Support Vector Classifier
    svc = SVC(C=10, gamma=0.1)  # Using the best parameters found previously
    classifier = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', svc)])
    classifier.fit(X_train, y_train)
    
    # Save the trained model for future use
    joblib.dump(classifier, 'svc_model.pkl')




if __name__ == "__main__":
    team = 'All'
    # team = ['Sri Lanka']
    # team = ['New Zealand', 'Sri Lanka', 'Australia']

    df = fetch_data(team)
    print(df)
    transformed_df = transform_data(df)
    # print(transformed_df)
    # print(transformed_df.columns)
    models_evaluate(transformed_df)
    # model_implement(transformed_df)

Connected to MySQL database
            team_1       team_2       winner     margin        ground  \
0     South Africa  New Zealand  New Zealand  5 wickets  Johannesburg   
1          England    Australia      England   100 runs   Southampton   
2      New Zealand    Australia    Australia    44 runs      Auckland   
3      New Zealand    Sri Lanka  New Zealand  5 wickets      Auckland   
4      New Zealand    Sri Lanka    Sri Lanka    18 runs    Wellington   
...            ...          ...          ...        ...           ...   
2573   New Zealand     Pakistan  New Zealand    45 runs       Dunedin   
2575         India  Afghanistan        India  6 wickets        Indore   
2577   New Zealand     Pakistan  New Zealand    21 runs      Hamilton   
2578   New Zealand     Pakistan  New Zealand    46 runs      Auckland   
2579         India  Afghanistan        India  6 wickets        Mohali   

        match_date     match_id  team_1_score  team_1_wicket team_1_over  \
0     Oct 21, 2005 